In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Load data

train_data = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')

In [ ]:
#Split the target value from dataset and drop id since we won't use it for predicting

X = train_data.drop(['id','target'], axis = 1)
y = train_data.target

In [ ]:
#Split data into training data and validation data

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 1, test_size = 0.4, train_size = 0.6)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score

In [ ]:
#Set the model

model = keras.Sequential([
    layers.Dense(units = 400, activation = 'relu', input_shape = [100]),
    layers.Dense(units = 400, activation = 'relu'),
    layers.Dense(units = 400, activation = 'relu'),
    layers.Dense(units = 1, activation = 'sigmoid'),
])

model.compile(
    optimizer = 'adam',
    loss = 'BinaryCrossentropy',
    metrics = ['AUC']
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta = 0.0001,
    patience = 20,
    restore_best_weights = True
)

In [ ]:
#Fit the data into the model

model.fit(X_train, y_train,
          validation_data =(X_valid, y_valid),
          batch_size = 64,
          epochs = 100,
          callbacks = [early_stopping],
         )

In [ ]:
#drop the id from the data before predicting

index = test_data.id
X_test = test_data.drop(['id'], axis = 1)
predictions = model.predict(X_test)

In [ ]:
#Put the id into predictions and save the predictions as CSV
predictions_df = pd.DataFrame(predictions, columns = ['target'])
output = pd.concat([index, predictions_df], axis = 1)
output.to_csv('my_submission.csv', index=False)
